# Использование функций GigaChat
В этом ноутбуке мы дадим возможность GigaChat вызывать внешние функции, 
а именно поиск в интернете с помощью [Tavily](https://tavily.com/)

## Установка
### Зависимости
```bash
npm install --save langchain langchain-gigachat @langchain/community @langchain/core
```
### Переменные окружения
```bash
export GIGACHAT_CREDENTIALS='ваши креды'
export GIGACHAT_SCOPE='GIGACHAT_API_CORP' или 'GIGACHAT_API_PERS' или 'GIGACHAT_API_B2B'
export TAVILY_API_KEY='API ключ от tavily'
```
Ключ API Tavily можно получить [здесь](https://app.tavily.com/)

## Запуск Tavily tool
Для начала импортируем Tavily tool и проверим, как он работает

In [2]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tool = new TavilySearchResults({
  maxResults: 2,
  // ...
});

In [3]:
await tool.invoke({
    input: "Какая сейчас погода в Москве?",
  });

'[{"title":"Погода в Москве сейчас - узнайте погоду в реальном времени, Москва ...","url":"https://weather.rambler.ru/v-moskve/now/","content":"Погода в Москве сейчас: облачно. Температура воздуха -2°С, относительная влажность воздуха 83%, атмосферное давление 753 мм. р.т., ветер северо-западный 1 м/с.","score":0.9035075,"raw_content":null},{"title":"Погода в Москве на сегодня подробно, прогноз погоды на сегодня, Москва ...","url":"https://weather.rambler.ru/v-moskve/today/","content":"Сегодня погода в Москве: облачно. Температура воздуха днём -2°С, вечером -2°С, ночью -4°С. Относительная влажность воздуха 84%, атмосферное давление 753 мм. р.т., ветер северо-западный 2 м/с.","score":0.85186684,"raw_content":null}]'

## Подключим LLM

In [4]:
import { Agent } from 'node:https';
import { GigaChat } from "langchain-gigachat"
/*
В обычном Node.js такое отключение проверки сертификатов срабатывает.

Но в Deno нет, 
поэтому используйте проставление NODE_EXTRA_CA_CERTS в env.
 */
const httpsAgent = new Agent({
    rejectUnauthorized: false,
}); 

const llm = new GigaChat({
    maxRetries: 0,
    httpsAgent
})

In [5]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant."],
  ["placeholder", "{messages}"],
]);

const llmWithTools = llm.bindTools([tool], {tool_choice: 'auto'});

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(async (userInput: string, config) => {
  const humanMessage = new HumanMessage(userInput);
  const aiMsg = await chain.invoke(
    {
      messages: [new HumanMessage(userInput)],
    },
    config
  );
  const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
  return chain.invoke(
    {
      messages: [humanMessage, aiMsg, ...toolMsgs],
    },
    config
  );
});

const toolChainResult = await toolChain.invoke(
  "Какая сейчас погода в Москве?"
);

UPDATE TOKEN


In [6]:
const { tool_calls, content } = toolChainResult;

console.log(
  "AIMessage",
  JSON.stringify(
    {
      tool_calls,
      content,
    },
    null,
    2
  )
);

AIMessage {
  "tool_calls": [],
  "content": "Сейчас в Москве облачно, температура воздуха -2°С, относительная влажность воздуха 83%, атмосферное давление 753 мм. р.т., ветер северо-западный 1 м/с."
}
